In [1]:
!pip install tqdm

In [2]:
import pandas as pd
import os
from tqdm import tqdm

# Mapeamento dos arquivos com colunas específicas
arquivos_txt = {
    "equipe": {
        "path": r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data\equipe.txt",
        "colunas": ["id_producao", "id_pessoa", "papel"],
        "encoding": "cp1252"  
    },
    "pessoa": {
        "path": r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data\pessoa.txt",
        "colunas": ["id_pessoa", "nome"],
        "encoding": "cp1252"
    },
    "producao": {
        "path": r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data\producao.txt",
        "colunas": ["id_producao", "titulo", "ano", "tipo_id"],
        "encoding": "cp1252"
    }
}

# Pasta de saída dos arquivos .parquet
pasta_saida = r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data"
os.makedirs(pasta_saida, exist_ok=True)

def converter_txt_para_parquet(nome, caminho_txt, colunas, encoding):
    try:
        with open(caminho_txt, mode="r", encoding=encoding, errors="replace") as f:
            df = pd.read_csv(
                f,
                sep="##",
                header=None,
                names=colunas,
                engine="python",
                on_bad_lines='skip'
            )
        caminho_parquet = os.path.join(pasta_saida, f"{nome}.parquet")
        df.to_parquet(caminho_parquet, engine="fastparquet", index=False)
        print(f"Ok {nome}.parquet criado com {len(df)} registros (com fallback de caracteres).")
    except Exception as e:
        print(f"[ERRO] ao converter {nome}: {e}")

# Execução
print("Iniciando conversão para PARQUET...")
for nome, meta in tqdm(arquivos_txt.items(), desc="Progresso"):
    print(f"\nConvertendo {nome}.txt...")
    converter_txt_para_parquet(nome, meta["path"], meta["colunas"], meta["encoding"])

Iniciando conversão para PARQUET...


Progresso:   0%|          | 0/3 [00:00<?, ?it/s]


Convertendo equipe.txt...
Ok equipe.parquet criado com 12849041 registros (com fallback de caracteres).


Progresso:  33%|███▎      | 1/3 [00:51<01:42, 51.42s/it]


Convertendo pessoa.txt...


Progresso:  67%|██████▋   | 2/3 [00:58<00:25, 25.36s/it]

Ok pessoa.parquet criado com 2062495 registros (com fallback de caracteres).

Convertendo producao.txt...


Progresso: 100%|██████████| 3/3 [01:03<00:00, 21.01s/it]

Ok producao.parquet criado com 1023616 registros (com fallback de caracteres).


Análise dos Dados


In [3]:
import pandas as pd
import os

# Caminho da pasta
pasta = r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data"

# Lê os arquivos
df_equipe = pd.read_parquet(os.path.join(pasta, "equipe.parquet"), engine="fastparquet")
df_pessoa = pd.read_parquet(os.path.join(pasta, "pessoa.parquet"), engine="fastparquet")
df_producao = pd.read_parquet(os.path.join(pasta, "producao.parquet"), engine="fastparquet")

# Função auxiliar de resumo
def resumo_df(nome, df):
    print(f"\n{'='*60}\n{nome.upper()} - {len(df)} registros\n{'='*60}")
    print("Colunas:", list(df.columns))
    print("\nTipos de dados:\n", df.dtypes)
    print("\nPrimeiras linhas:\n", df.head())
    print("\nEstatísticas gerais:\n", df.describe(include='all'))
    print("\nValores nulos por coluna:\n", df.isnull().sum())
    print("\nDuplicatas:", df.duplicated().sum())

pd.set_option("display.max_columns", None)         
pd.set_option("display.max_rows", 100)             
pd.set_option("display.max_colwidth", None)        
pd.set_option("display.expand_frame_repr", False)   

# Análise individual
resumo_df("equipe", df_equipe)
resumo_df("pessoa", df_pessoa)
resumo_df("producao", df_producao)


EQUIPE - 12849041 registros
Colunas: ['id_producao', 'id_pessoa', 'papel']

Tipos de dados:
 id_producao     int64
id_pessoa       int64
papel          object
dtype: object

Primeiras linhas:
    id_producao  id_pessoa            papel
0       715190      37497  Nadine's Client
1       715190     124692             None
2       715190     258179         Reporter
3       715190     516564       The Butler
4       715190     683371     Palace Guard

Estatísticas gerais:
          id_producao     id_pessoa    papel
count   1.284904e+07  1.284904e+07  6458012
unique           NaN           NaN  1893157
top              NaN           NaN  Himself
freq             NaN           NaN   432721
mean    8.624120e+05  4.542122e+05      NaN
std     5.413430e+05  2.881670e+05      NaN
min     1.000000e+00  1.000000e+00      NaN
25%     3.974140e+05  2.091830e+05      NaN
50%     8.045930e+05  4.199990e+05      NaN
75%     1.313009e+06  6.918340e+05      NaN
max     2.012386e+06  1.017525e+06      N

In [4]:
import pandas as pd
import os

# Caminho da pasta onde estão os arquivos .parquet
pasta = r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data"

# Lista de arquivos a carregar
arquivos = {
    "equipe": "equipe.parquet",
    "pessoa": "pessoa.parquet",
    "producao": "producao.parquet"
}

# Leitura e exibição
for nome, arquivo in arquivos.items():
    caminho = os.path.join(pasta, arquivo)
    df = pd.read_parquet(caminho, engine="fastparquet")
    print(f"\n{'='*60}\n{nome.upper()} - 5 primeiras linhas\n{'='*60}")
    print(df.head())


EQUIPE - 5 primeiras linhas
   id_producao  id_pessoa            papel
0       715190      37497  Nadine's Client
1       715190     124692             None
2       715190     258179         Reporter
3       715190     516564       The Butler
4       715190     683371     Palace Guard

PESSOA - 5 primeiras linhas
   id_pessoa             nome
0       2523    Accardi, A.J.
1       3205     Acuna, Ramón
2       2725  Achard, Olivier
3        156        A., David
4        360     Aaron, James

PRODUCAO - 5 primeiras linhas
   id_producao               titulo   ano  tipo_id
0        70625  Campanile d'oro, Il  1955        1
1        76172          Caída libre  1994        1
2        97851     Crucea de piatra  1993        1
3        86809          Clinic, The  2007        1
4        79563       Chauffeur, The  1921        1


Remover Ano = 0

In [5]:
import pandas as pd
import os

# Caminho
pasta = r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data"
caminho_original = os.path.join(pasta, "producao.parquet")
caminho_filtrado = os.path.join(pasta, "producao_filtrado.parquet")

# Carrega o DataFrame
df = pd.read_parquet(caminho_original, engine="fastparquet")

# Mostra os tipos originais
print("Antes da limpeza, tipos:\n", df.dtypes)

# Converte para inteiro (trata strings)
df["ano"] = pd.to_numeric(df["ano"], errors="coerce").fillna(0).astype(int)

# Mostra valores únicos antes da filtragem
print("\nValores únicos antes da limpeza:\n", sorted(df["ano"].unique()))

# Filtra
df_filtrado = df[df["ano"] != 0]

# Relatório
print(f"\nRegistros antes: {len(df)}")
print(f"Registros após remover ano = 0: {len(df_filtrado)}")
print(f"Removidos: {len(df) - len(df_filtrado)}")

# Mostra valores únicos após limpeza
print("\nValores únicos após limpeza:\n", sorted(df_filtrado["ano"].unique()))

# Salva
df_filtrado.to_parquet(caminho_filtrado, index=False, engine="fastparquet")
print(f"\nArquivo salvo em: {caminho_filtrado}")

Antes da limpeza, tipos:
 id_producao     int64
titulo         object
ano            object
tipo_id         int64
dtype: object

Valores únicos antes da limpeza:
 [0, 1519, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]

Registros antes: 1023616
Registros após remover ano = 0: 845867
Removi

In [6]:
print(df_filtrado["ano"].eq(0).sum())  
print(sorted(df_filtrado["ano"].unique()))  

0
[1519, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]


In [7]:
import pandas as pd

df["ano"] = pd.to_numeric(df["ano"], errors="coerce")

tem_zero = (df["ano"] == 0).sum()
tem_nan = df["ano"].isna().sum()

print(f"→ Quantidade de registros com ano = 0: {tem_zero}")
print(f"→ Quantidade de registros com ano = NaN: {tem_nan}")

→ Quantidade de registros com ano = 0: 177749
→ Quantidade de registros com ano = NaN: 0


In [8]:
import pandas as pd

# Caminho do arquivo original
caminho = r"C:\Users\Willgnner\Documents\dba_postgresql\homework\data\producao.txt"

# Lendo o arquivo com codificação apropriada
df = pd.read_csv(caminho, sep="##", header=None, engine="python", encoding="latin1")

# Mostrando as colunas detectadas
print("Número de colunas detectadas:", df.shape[1])
print("Colunas:")
for i, col in enumerate(df.columns):
    print(f" - Coluna {i}: nome sugerido = 'col_{i}'")

# Exibindo as primeiras 5 linhas
print("\nPrimeiras linhas:")
print(df.head())

Número de colunas detectadas: 4
Colunas:
 - Coluna 0: nome sugerido = 'col_0'
 - Coluna 1: nome sugerido = 'col_1'
 - Coluna 2: nome sugerido = 'col_2'
 - Coluna 3: nome sugerido = 'col_3'

Primeiras linhas:
       0                    1     2  3
0  70625  Campanile d'oro, Il  1955  1
1  76172          Caída libre  1994  1
2  97851     Crucea de piatra  1993  1
3  86809          Clinic, The  2007  1
4  79563       Chauffeur, The  1921  1
